# La transformada cuántica de Fourier

## Ejemplo de transformada de Fourier

De acuerdo a la documentación de Julia [http://docs.julialang.org/en/release-0.5/stdlib/math/], la formula para la transformada es 
\begin{equation}
\operatorname{DFT}(A)[k] =
  \sum_{n=1}^{\operatorname{length}(A)}
  \exp\left(-i\frac{2\pi
  (n-1)(k-1)}{\operatorname{length}(A)} \right) A[n].
\end{equation}
De hecho podemos hacer la relación con la transformada continua, y vemos como estamos simplemente sacando las componentes planas del vector $A$ al vector $\operatorname{DFT}(A)$.

In [ ]:
using PyPlot

In [ ]:
f = [abs(x) <= 1 ? 1 : 0 for x in -5:0.1:5];
F = fft(f);

In [ ]:
plotf=plot(f, color="red", linewidth=2.0);

In [ ]:
plotF=plot(abs(F), color="red", linewidth=2.0);

## Carga de archivos básicos

In [ ]:
# Pkg.add("LsqFit")

In [ ]:
using PyPlot
push!(LOAD_PATH, ".");
using quantum
names(quantum)

## Chop function

Comenzaremos por hacer una función que es útil para hacer debugin y evitar la basura en pantalla. Esta me aproxima cosas cercanas a cero, a un cero exacto. 

Para esto, vamos a utilizar funciones en las que se especifica el *tipo de entrada* de la función. Esto es un aspecto en que podemos enfatizar para optimizar nuestras rutinas. Las palabras claves son *type stability*. Para esto, usamos algo como 
~~~
x::Float64
~~~
de manera que la definición solo aplica cuando el argumento es del tipo Float64. 

De manera similar, vamos a especificar la salida de la función, requiriendo que nuestro "0" sea un cero con varios puntos decimales y así evitar mezclar tipos. 

Finalmente, usaremos el concepto de *function overloading* que nos permite dar a varias funciones el mismo nombre, siempre que estas estén especificadas sobre diferentes tipos de variables. Así, al llamar la función, julia puede saber cual versión llamar, dependiendo del argumento.

In [ ]:
function chopnumber(x::Float64, epsilon=10e-14)   
    if abs(x) < epsilon
        return Float64(0)
    else
        return x
    end
end

function chopnumber(x::Complex{Float64}, epsilon=10e-14)   
    return chopnumber(real(x),epsilon)+im*chopnumber(imag(x),epsilon)
end

function chopnumber(y::Array{Complex{Float64},1}, epsilon=10e-14) 
    x=copy(y)
    for i=1:length(x)
        x[i]=chopnumber(x[i],epsilon)
    end
    return x
end

function chopnumber(y::Array{Complex{Float64},2}, epsilon=10e-14) 
    x=copy(y)
    for i=1:length(x)
        x[i]=chopnumber(x[i],epsilon)
    end
    return x
end

## Hadamard, phase gate, and controlled phase gate

La compuerta Hadamard se usa para crear superposiciones. En particular, actúa como 
\begin{align}
H|0\rangle &= \frac{|0\rangle + |1\rangle}{\sqrt{2}}\\
H|1\rangle &= \frac{|0\rangle - |1\rangle}{\sqrt{2}}
\end{align}
por lo que su representación matricial es simplemente
\begin{equation}
H=
\frac{1}{\sqrt{2}}
\begin{pmatrix}
1 & 1 \\
1 & -1
\end{pmatrix}.
\end{equation}

In [ ]:
hadamard=[1 1; 1 -1]/sqrt(2);

Se usa con frecuencia para crear una superposición de todos los estados de la base computacional. 

Crearemos una función que nos permitirá construir matrices de manera simple. Es la función que me general la representación vectorial de $|n\rangle$ en la base computacional.

In [ ]:
function base_state(i,dim)
    psi=zeros(Complex{Float64},dim)
    psi[i+1]=1;
    return psi
end

Podemos crear una superposición de todos los elementos de la base con una Hadamard sobre cada uno de los qubits:

In [ ]:
n=3
psi=base_state(0,2^n)
for i=0:n-1
    apply_unitary!(psi,hadamard,i)
end
psi

Ahora vamos a estudiar el _phase gate_, que es simplemente añadir una fase a un qubit, si este esta en el estado $|1\rangle$. Es decir, es 
\begin{equation}
\begin{pmatrix}
1 & 0 \\
0 & \text{exp}(i \phi)
\end{pmatrix}
\end{equation}.

En nuestro caso, estaremos interesados en la versión controlada. Es decir, si el qubit de control está en el estado $|1\rangle$ aplicamos la compuerta, de lo contrario no la aplicamos. En caso de tener un _controled_-$u$, la operación que debemos aplicar sobre los dos qubits es 
\begin{equation}
1\oplus u
\end{equation}
y no $1\otimes u$. En el caso particular del _controled phase_, obtendríamos 
\begin{equation}
\begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 &\text{exp}(i \phi)
\end{pmatrix}
\end{equation}.

Para implementarla, nótese que su acción sobre dos qubits es no trivial solo sobre el estado $|11\rangle$, al que le aplica una fase. Usamos la función
~~~
testbit
~~~
y la operación AND, que es codificada en Julia como
~~~
&.
~~~

In [ ]:
function apply_control_phase!(psi, control_qubit, target_qubit,angle)
    phase=exp(im*angle)
    for i = 0: length(psi)-1
        if testbit(i,control_qubit) & testbit(i,target_qubit)
            psi[i+1]*=phase
        end
    end
end 

## La transformada cuántica de Fourier [del libro de Nielsen y Chuang]

Supongamos que tenemos un vector $\vec x$ con componentes $x_j$. Entonces, la transformada de Fourier discreta de dicho vector es 
$$
y_k =  \frac{1}{\sqrt{N}} \sum_{j=0}^{N-1} x_j \text{exp}
\big(
\frac{2\pi \text{i} j k}{N}
\big).
$$

Sea $U_\text{QFT}$ el operador transformada de Fourier. La definiremos como 
$$
U_\text{QFT} \sum_{j=0}^{N-1} x_j |j\rangle
= 
 \sum_{k=0}^{N-1} y_k |k\rangle
$$
con el vector $\vec y $ dado por la transformada discreta de Fourier arriba mencionada.
Haciendo que los estados transformados sean justamente los de la base computacional, tenemos la identidad
$$
U_\text{QFT} |j\rangle =  \frac{1}{\sqrt{N}} \sum_{k=0}^{N-1} \text{exp}
\big(
\frac{2\pi \text{i} j k}{N}
\big) |k\rangle.
$$

Para mostrar el circuito que realiza esta expresión, conviene dar una fórmula alternativa para la transformada de Fourier. Esta se basa en expresar algunos números en notación binaria. 
\begin{align}
j &\to \frac{1}{2^{n/2}} \sum_{k=0}^{2^n-1} \text{exp} \left(\frac{ 2\pi \text{i} jk}{2^n} \right) |k\rangle \\
&= \frac{1}{2^{n/2}} \sum_{k_1=0}^1 \cdots \sum_{k_n=0}^1 \text{exp} 
  \left[2\pi \text{i} j \left(\sum_{l=1}^n k_l 2^{-l} \right) \right] |k_1 \dots k_n \rangle \\
&= \frac{1}{2^{n/2}} \sum_{k_1=0}^1 \cdots \sum_{k_n=0}^1 \bigotimes_{l=1}^n
   \text{exp} 
  \left[2\pi \text{i} j k_l 2^{-l} \right] |k_l \rangle \\
&= \frac{1}{2^{n/2}}  \bigotimes_{l=1}^n
   \sum_{k_l=0}^1
   \text{exp} 
  \left[2\pi \text{i} j k_l 2^{-l} \right] |k_l \rangle \\
&= \frac{1}{2^{n/2}}  \bigotimes_{l=1}^n
   \left(|0\rangle +
   \text{exp} 
  \left[2\pi \text{i} j 2^{-l} \right] |1 \rangle \right) \\
&= \frac{
\left( |0\rangle + e^{2\pi \text{i} 0.j_n} |1\rangle \right)
\otimes
\left( |0\rangle + e^{2\pi \text{i} 0.j_{n-1}j_n} |1\rangle \right)
\otimes
\cdots
\otimes
\left( |0\rangle + e^{2\pi \text{i} 0.j_1 j_2 \cdots j_n} |1\rangle \right)
}{2^{n/2}}
\end{align}

1. La dimensión de mi sistema es $N = 2^n$.
2. Separamos $k/2^n$ como un _string_ de ceros y unos, tanto en el exponente como en el ket.
3. Notamos que el producto tensorial también es producto entre complejos, y bajamos la suma en el exponente como un producto tensorial.
4. Factorizamos las sumas [suma y productos conmutan]
5. Desarrollamos la suma interna
6. Desarrollamos cada uno de los términos, teniendo en cuenta que la parte entera del exponente, después del $2\pi$ no contribuye (y por lo tanto la ignoramos).

El circuito estará compuesto por 
* compuertas Hadamard, para crear las superposiciones en cada qubit
* _phase gates_ para poner la fase en el $|1\rangle$ de cada qubit
* controladas, por los otros qubits. 


![Alt](qft_circuit.png "Title")

La compuerta controlada $R_k$ es la _controlled phase gate_ por una potencia inversa de 2:
\begin{equation}
R_k = 
\begin{pmatrix}
1 & 0 \\
0& e^{2\pi \text{i}/2^k}
\end{pmatrix}
\end{equation}

Estudiaremos ahora que sucede cuando tomamos un estado de la base computacional 
\begin{equation}
|j_1 j_2 \dots j_n\rangle
\end{equation}
como entrada para el circuito cuántico. Note que
\begin{equation}
0.j_1 = \begin{cases}
0 & \text{if } j_1= 0 \\
1/2 & \text{if } j_1= 1
\end{cases}
\end{equation}
por lo que 
\begin{equation}
\text{exp}(2\pi \text{i} 0.j_1) = \begin{cases}
1 & \text{if } j_1= 0 \\
-1 & \text{if } j_1= 1
\end{cases}
\end{equation}
y entonces el estado, después de aplicar el primer Hadamard es 
\begin{equation}
\frac{1}{\sqrt{2}}\left(|0\rangle + e^{2\pi \text{i} 0.j_1}|1\rangle\right)\otimes
|j_2 j_3 \dots j_n\rangle.
\end{equation}
Al aplicar el $R_2$ controlado por el qubit 2 (en esta numeración), aplicaremos una fase 
$e^{2\pi \text{i}/2^k}=e^{2\pi \text{i}0.0j_2}$ al primer qubit, cuando está en uno. Así, el nuevo estado será
\begin{equation}
\frac{1}{\sqrt{2}}\left(|0\rangle + e^{2\pi \text{i} 0.j_1 j_2}|1\rangle\right)\otimes
|j_2 j_3 \dots j_n\rangle.
\end{equation}
Continuando con ese proceso, al finalizar esa secuencia de controlled R's, tendremos el siguiente estado
\begin{equation}
\frac{1}{\sqrt{2}}\left(|0\rangle + e^{2\pi \text{i} 0.j_1 j_2\cdots j_n}|1\rangle\right)\otimes
|j_2 j_3 \dots j_n\rangle.
\end{equation}

Repetimos el procedimiento con el siguiente qubit y tendremos, despues del Hadamard y los controlled gates, el estado
\begin{equation}
\frac{1}{2^{2/2}}
\left(|0\rangle + e^{2\pi \text{i} 0.j_1 j_2\cdots j_n}|1\rangle\right)\otimes
\left(|0\rangle + e^{2\pi \text{i} 0.j_2 j_3\cdots j_n}|1\rangle\right)\otimes
|j_3 j_4 \dots j_n\rangle.
\end{equation}
Despues de hacer lo mismo con todos los qubits, tendremos el estado 
\begin{equation}
\frac{1}{2^{n/2}}
\left(|0\rangle + e^{2\pi \text{i} 0.j_1 j_2\cdots j_n}|1\rangle\right) \otimes 
\left(|0\rangle + e^{2\pi \text{i} 0.j_2 j_3\cdots j_n}|1\rangle\right)\otimes
\left(|0\rangle + e^{2\pi \text{i} 0.j_3 j_4\cdots j_n}|1\rangle\right) \otimes\cdots\otimes
\left(|0\rangle + e^{2\pi \text{i} 0.j_n}|1\rangle\right).
\end{equation}
Lo único que hace falta para tener la transformada de Fourier del estado es hacer un _swap_ de todos los qubits. Esto son sólo $n/2$ compuertas. Finalmente, tenemos el estado transformada de Fourier. 
\begin{equation}
 \frac{
\left( |0\rangle + e^{2\pi \text{i} 0.j_n} |1\rangle \right)
\otimes
\left( |0\rangle + e^{2\pi \text{i} 0.j_{n-1}j_n} |1\rangle \right)
\otimes
\cdots
\otimes
\left( |0\rangle + e^{2\pi \text{i} 0.j_1 j_2 \cdots j_n} |1\rangle \right)
}{2^{n/2}}
\end{equation}


Para hacer el paso final, necesitamos hacer el swap gate. Acá de nuevo vamos a usar low-level bit programing. Comenzaremos por hacer un flip de un bit. Es decir, si el bit está en 0 lo pasamos a 1 y viceversa. De nuevo, usaremos una máscara para el bit 
~~~
 1 << bit
~~~
que tiene un 1 en el bit indicado y 0 en el resto. 

También usaremos el operador `$` que actúa como XOR. Al combinarlo con la mascara, cambia solamente el bit en cuestion. Así, creamos la función  `flip_bit`

In [ ]:
function flip_bit(number,bit)
    number $ (1 << bit)
end
flip_bit(5,2)

Ahora creamos la función `reverse_bits` que invierte el orden de los primeros `n` bits. Para esto vamos sobre la primera mitad de bits (usando la división entera `fld`). 

In [ ]:
function reverse_bits(i::Int,number_of_bits)
    number=i
    top_bit=fld(number_of_bits,2)
    for i=0:(top_bit-1)            
        if testbit(number,i) $ testbit(number,number_of_bits-1-i)
            number=flip_bit(number,i)
            number=flip_bit(number,number_of_bits-1-i)
        end
    end
    return number
end
#@show reverse_bits(5,4)

Para probar este código, tomamos los bits como un string y le hacemos un reverse. Para esto usamos funciones para la manipulacion de strings como `reverse` o al anteriormente usada `bin`

In [ ]:
n=3
for i=0:2^n-1
    @show [reverse(bin(i,n)) bin(reverse_bits(i,n),n)]
end
n=17
for i=0:2^n-1
    if ~(reverse(bin(i,n))==bin(reverse_bits(i,n),n))
        @show "error", i, n, bin(i,n)
    end
end

Ahora, la debemos aplicar al estado completo. Para esto, y para controlar que no hacemos el loop doble, vamos a verificar que los dos números que vamos a intercambiar (por ejemplo el `011` con el `110`) estén en orden. Esta función no la vamos a verificar, pues al hacer el debuging de la QFT, que es más estricto, vemos que debe estar bien. 

In [ ]:
function reverse_bits!(psi::Array{Complex{Float64},1})
    number_of_qubits = trailing_zeros(length(psi))
    for i=0:2^number_of_qubits-1
        i_reversed=reverse_bits(i,number_of_qubits)
        if i<i_reversed
            psi[i+1], psi[i_reversed+1] = psi[i_reversed+1], psi[i+1]
        end
    end
end

In [ ]:
function quantum_fourier!(psi)
    n= trailing_zeros(length(psi))
    for i=n-1:-1:0
        apply_unitary!(psi, hadamard, i)
        for j=i-1:-1:0
            r_order=i-j+1;
            angle=2*pi/2.^r_order
            apply_control_phase!(psi,j,i,angle)
        end
    end
    reverse_bits!(psi)
end

Podemos construir las matrices que corresponden a las transformaciones lineales QFT y IFFT. Al compararlas, vemos que son la misma (módulo un factor de normalización). De hecho, podemos ver que su estructura es simplemente
\begin{equation}
\begin{pmatrix}
1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 \\
1 & \omega^1 & \omega^2 & \omega^3 & \omega^4 & \omega^5 & \omega^6 & \omega^7 \\
1 & \omega^2 & \omega^4 & \omega^6 & 1 & \omega^2 & \omega^4 & \omega^6 \\
1 & \omega^3 & \omega^6 & \omega^1 & \omega^4 & \omega^7 & \omega^2 & \omega^5 \\
1 & \omega^4 & 1 & \omega^4 & 1 & \omega^4 & 1 & \omega^4 \\
1 & \omega^5 & \omega^2 & \omega^7 & \omega^4 & \omega^1 & \omega^6 & \omega^3 \\
1 & \omega^6 & \omega^4 & \omega^2 & 1 & \omega^6 & \omega^4 & \omega^2 \\
1 & \omega^7 & \omega^6 & \omega^5 & \omega^4 & \omega^3 & \omega^2 & \omega^1
\end{pmatrix}
=
\begin{pmatrix}
1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 \\
1 & \omega^1 & \omega^2    & \omega^3    & \omega^4    & \omega^5    & \omega^6    & \omega^7 \\
1 & \omega^2 & \omega^4    & \omega^6    & \omega^8    & \omega^{10} & \omega^{12} & \omega^{14} \\
1 & \omega^3 & \omega^6    & \omega^9    & \omega^{12} & \omega^{15} & \omega^{18} & \omega^{21} \\
1 & \omega^4 & \omega^8    & \omega^{12} & \omega^{16} & \omega^{20} & \omega^{24} & \omega^{28} \\
1 & \omega^5 & \omega^{10} & \omega^{15} & \omega^{20} & \omega^{25} & \omega^{30} & \omega^{35} \\
1 & \omega^6 & \omega^{12} & \omega^{18} & \omega^{24} & \omega^{30} & \omega^{36} & \omega^{42} \\
1 & \omega^7 & \omega^{14} & \omega^{21} & \omega^{28} & \omega^{35} & \omega^{42} & \omega^{49}
\end{pmatrix}
=\omega^{ij}
\end{equation}
con 
\begin{equation}
\omega= \text{e}^\frac{2\pi \text{i}}{8}
\end{equation}


In [ ]:
n=5;
dim=2^n
ui=zeros(Complex{Float64},dim,dim)
uq=zeros(Complex{Float64},dim,dim)
for i=0:(dim-1)
    ui[:,i+1]=ifft(base_state(i,dim))*2^(n/2)
    psi=base_state(i,dim);
    quantum_fourier!(psi)
    uq[:,i+1]=psi
end

w=exp(2*π*im/dim)
Ap = Complex{Float64}[ w^(i*j) for i=0:dim-1, j=0:dim-1]/sqrt(dim);
@show [ norm(chopnumber(uq-ui)) norm(uq-Ap)];

[De la wikipedia]

In quantum computing, the quantum Fourier transform is a linear transformation on quantum bits, and is the quantum analogue of the discrete Fourier transform. The quantum Fourier transform is a part of many quantum algorithms, notably Shor's algorithm for factoring and computing the discrete logarithm, the quantum phase estimation algorithm for estimating the eigenvalues of a unitary operator, and algorithms for the hidden subgroup problem.

The quantum Fourier transform can be performed efficiently on a quantum computer, with a particular decomposition into a product of simpler unitary matrices. Using a simple decomposition, the discrete Fourier transform on $2^{n}$￼ amplitudes can be implemented as a quantum circuit consisting of only $O(n^{2})$￼ Hadamard gates and controlled phase shift gates, where $n$￼ is the number of qubits. This can be compared with the classical discrete Fourier transform, which takes $O(n2^{n})$￼ gates (where $n$￼ is the number of bits), which is exponentially more than $O(n^{2})$￼. However, the quantum Fourier transform acts on a quantum state, whereas the classical Fourier transform acts on a vector, so not every task that uses the classical Fourier transform can take advantage of this exponential speedup.

The best quantum Fourier transform algorithms known today require only $O(n\log n)$￼ gates to achieve an efficient approximation


# Otros posibles sistemas y comentarios para la siguiente iteracion

* Recolectar las rutinas en el modulo quantum, construirlas aca poco a poco, luego, al llamarlas, poner el código en pantalla.
* Hacer un pequeño comentario de para que sirve la transformada cuantica de Fourier.
* Revisar que el paquete quantum tenga LsqFit como un requisito desde adentro.
* Agregar las funciones que tenemos acá e irlas construyendo y luego mostrar el código que tenemos dentro del paquete.
* Hacer una función que me permita meterle una función anónima como argumento y que me bote la representación matricial de un operador arbitrario.